In [1]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, Dataset, DataLoader
from model import ViT
from torch.optim import lr_scheduler

from tqdm import tqdm
import numpy as np
from random import randint
import random
import warmup_scheduler


In [2]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [3]:
hp = {
    'batch_size': 256,
    'num_workers': 4,
    'seed': 42
}
seed = hp['seed']
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [4]:
cifar10_mean = [0.4914, 0.4822, 0.4465]
cifar10_std = [0.2470, 0.2435, 0.2616]

cifar10_path = '../data/vision/cifar10/'
cifar10_train_transform = transforms.Compose(
            [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(cifar10_mean, cifar10_std),
            ]
        )
cifar10_test_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(cifar10_mean, cifar10_std),
            ]
        )
cifar10_train = CIFAR10(cifar10_path, train=True, transform=cifar10_train_transform)
cifar10_test = CIFAR10(cifar10_path, train=False, transform=cifar10_test_transform)

batch_size = hp['batch_size']
num_workers = hp['num_workers']
train_dataloader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_dataloader = DataLoader(cifar10_test, batch_size=512, shuffle=False, num_workers=num_workers, drop_last=False)

In [5]:
def train(model, train_dataloader, val_dataloader, optimizer, scheduler, loss_fn, epochs):
    optimizer.zero_grad()
    optimizer.step()
    for epoch in range(epochs):    
        scheduler.step(epoch + 1)
        train_epoch(model, train_dataloader, optimizer, loss_fn)
        val_epoch(model, val_dataloader, loss_fn)


In [6]:
def train_epoch(model, train_dataloader, optimizer, loss_fn):
    # step variable
    epoch_total_loss = 0
    epoch_steps = 0
    sample_size = 0
    correct = 0
    
    patch_num = 64
    #set train mode
    model.train()
    for batch_idx, (inputs, labels) in tqdm(enumerate(train_dataloader)):
        # backprop
        optimizer.zero_grad()
        inputs, labels = inputs.cuda(), labels.cuda()
        if model.joint:
            outputs = model(inputs[::2], inputs[1::2])
            labels = labels.repeat_interleave(patch_num)
        else:
            outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        sample_size += labels.size(0)
        
        # update step
        epoch_total_loss += loss.item()
        epoch_steps += 1
        # change learning rate


    epoch_loss = epoch_total_loss / epoch_steps
    accuracy = correct / sample_size

    print(f"learning_rate ${optimizer.param_groups[0]['lr']}")
    print(f"train_epoch_loss ${epoch_loss}")
    print(f"training_accuracy ${accuracy}")
    return epoch_steps

In [7]:
def val_epoch(model, val_dataloader, loss_fn):
    # step_variable
    epoch_total_loss = 0
    epoch_steps = 0
    sample_size = 0
    correct = 0
    
    # set eval mode
    model.eval()
    for batch_idx, (inputs, labels) in tqdm(enumerate(val_dataloader, 0)):
        with torch.no_grad():
            # inference
            inputs, labels = inputs.cuda(), labels.cuda()
            if model.joint:
                outputs = model(inputs, inputs)
            else:
                outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            sample_size += labels.size(0)
            epoch_total_loss += loss.item()
            epoch_steps += 1
    epoch_loss = epoch_total_loss / epoch_steps
    accuracy = correct / sample_size
    print(f"validation_loss ${epoch_loss}")
    print(f"accuracy ${accuracy}")

In [8]:
d_model = 128
head = 8
d_ff = 1024
dropout = 0
encoder_layers = nn.TransformerEncoderLayer(d_model, head, d_ff, dropout, batch_first=True)
transformer = nn.TransformerEncoder(encoder_layers, 6)
model = ViT(32, 4, d_model, transformer, 10, joint=True)
model.cuda()

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=4, p2=4)
    (1): Linear(in_features=48, out_features=128, bias=True)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(i

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
warm_up_epoch = 5
epochs = 200
base_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler = warmup_scheduler.GradualWarmupScheduler(optimizer, multiplier=1., total_epoch=warm_up_epoch, after_scheduler=base_scheduler)

train(model, train_dataloader, val_dataloader, optimizer, scheduler, loss_fn, epochs)

/home/terry/miniconda3/envs/research/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
195it [00:08, 23.33it/s]

learning_rate $0.0002
train_epoch_loss $2.1525448041084485
training_accuracy $0.1882114508213141



20it [00:01, 18.83it/s]

validation_loss $1.900147420167923
accuracy $0.2964



195it [00:07, 25.03it/s]

...

validation_loss $0.5398990288376808
accuracy $0.8573



195it [00:07, 24.89it/s]

learning_rate $1.541333133436018e-06
train_epoch_loss $0.03264300668946443
training_accuracy $0.9883936172876603



20it [00:01, 18.70it/s]

validation_loss $0.5399981454014778
accuracy $0.8573


In [10]:
model

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=4, p2=4)
    (1): Linear(in_features=48, out_features=128, bias=True)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(i